In [21]:
import requests
import sqlite3
import time


lat = '51.5072'
lon = '0.1276'
now = int(time.time())
from key import key

link = f'https://api.openweathermap.org/data/3.0/onecall/timemachine?lat={lat}&lon={lon}&dt={now}&appid={key}&units=metric'
conn = sqlite3.connect('weather.db')
response = requests.get(link)
response.text

'{"lat":51.5072,"lon":0.1276,"timezone":"Europe/London","timezone_offset":0,"data":[{"dt":1708593601,"sunrise":1708585263,"sunset":1708622726,"temp":11.21,"feels_like":10.68,"pressure":984,"humidity":88,"dew_point":9.3,"uvi":0.13,"clouds":75,"visibility":10000,"wind_speed":5.14,"wind_deg":170,"weather":[{"id":803,"main":"Clouds","description":"broken clouds","icon":"04d"}]}]}'

In [4]:
data = response.json()
data

{'lat': 51.5072,
 'lon': 0.1276,
 'timezone': 'Europe/London',
 'timezone_offset': 0,
 'data': [{'dt': 1708590535,
   'sunrise': 1708585263,
   'sunset': 1708622726,
   'temp': 10.33,
   'feels_like': 9.77,
   'pressure': 986,
   'humidity': 90,
   'dew_point': 8.76,
   'uvi': 0.04,
   'clouds': 75,
   'visibility': 10000,
   'wind_speed': 5.66,
   'wind_deg': 170,
   'weather': [{'id': 500,
     'main': 'Rain',
     'description': 'light rain',
     'icon': '10d'}]}]}

In [5]:
d1 = {'lat': data['lat'], 'lon': data['lon'], 'timezone': data['timezone'], 'timezone_offset': data['timezone_offset']}

for i in data['data']:
    d1.update(i)

for i in d1['weather']:
    d1.update(i)

d1.pop('weather')

data = d1
data

{'lat': 51.5072,
 'lon': 0.1276,
 'timezone': 'Europe/London',
 'timezone_offset': 0,
 'dt': 1708590535,
 'sunrise': 1708585263,
 'sunset': 1708622726,
 'temp': 10.33,
 'feels_like': 9.77,
 'pressure': 986,
 'humidity': 90,
 'dew_point': 8.76,
 'uvi': 0.04,
 'clouds': 75,
 'visibility': 10000,
 'wind_speed': 5.66,
 'wind_deg': 170,
 'id': 500,
 'main': 'Rain',
 'description': 'light rain',
 'icon': '10d'}

In [23]:
l1 = []

for n, i in data.items():
    l1.append(i)

l1

[51.5072,
 0.1276,
 'Europe/London',
 0,
 1708590032,
 1708585263,
 1708622726,
 10.33,
 9.77,
 986,
 90,
 8.76,
 0.04,
 75,
 10000,
 5.66,
 170,
 500,
 'Rain',
 'light rain',
 '10d']

In [24]:
conn = sqlite3.connect('weather.db')
cursor = conn.cursor()

cursor.executescript('''

            CREATE TABLE IF NOT EXISTS weather (
                     lat,
                     long,
                     timezone,
                     timezone_offset,
                     date,
                     sunrise,
                     sunset,
                     temp,
                     feels_like,
                     pressure,
                     humidity,
                     dew_point,
                     uvi,
                     clouds,
                     visibility,
                     wind_speed,
                     wind_deg,
                     wind_gust,
                     weather_id,
                     main,
                     description,
                     icon     
        )
    ''')

cursor.execute('INSERT INTO weather VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)',
                        (l1))
conn.commit()

ProgrammingError: Incorrect number of bindings supplied. The current statement uses 22, and there are 21 supplied.

In [96]:
def drop_table():

    cursor = conn.cursor()

    cursor.executescript('''

        DROP TABLE IF EXISTS weather

    ''')

    conn.commit()

drop_table()

Abandoning ETL method as API only includes available data, leading to errors like:

"ProgrammingError: Incorrect number of bindings supplied. The current statement uses 22, and there are 21 supplied."

In [85]:
try:
    with open('last_executed.txt') as f:
        last_execution_date = int(f.read())
except FileNotFoundError:
        last_execution_date = int(time.time())

last_execution_date

1706000000

In [70]:
with open('last_executed.txt', 'w') as f:
            f.write(str(int(time.time())))

In [92]:
def get_links():
    if last_execution_date // 86400 ==  time.time() // 86400:
        return []

    links = []

    def build_link(date):
        lat = '51.5072'
        lon = '0.1276'
        from key import key
        link = f'https://api.openweathermap.org/data/3.0/onecall/timemachine?lat={lat}&lon={lon}&dt={date}&appid={key}&units=metric'
        links.append(link)

    nearest_hour = int(time.time() - time.time() % 3600)
    nearest_hour_old = last_execution_date - last_execution_date % 3600

    for x in range(nearest_hour_old, nearest_hour, 3600):
        build_link(x)
    
    return links

get_links()

['https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=51.5072&lon=0.1276&dt=1705996800&appid=e276140c3e33a20f93a7c86e198a285e&units=metric',
 'https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=51.5072&lon=0.1276&dt=1706000400&appid=e276140c3e33a20f93a7c86e198a285e&units=metric',
 'https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=51.5072&lon=0.1276&dt=1706004000&appid=e276140c3e33a20f93a7c86e198a285e&units=metric',
 'https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=51.5072&lon=0.1276&dt=1706007600&appid=e276140c3e33a20f93a7c86e198a285e&units=metric',
 'https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=51.5072&lon=0.1276&dt=1706011200&appid=e276140c3e33a20f93a7c86e198a285e&units=metric',
 'https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=51.5072&lon=0.1276&dt=1706014800&appid=e276140c3e33a20f93a7c86e198a285e&units=metric',
 'https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=51.5072&lon=0.12

In [86]:
nearest_hour = int(time.time() - time.time() % 3600)
nearest_hour_old = last_execution_date - last_execution_date % 3600

for x in range(nearest_hour_old, nearest_hour, 3600):
    print(x)

1705996800
1706000400
1706004000
1706007600
1706011200
1706014800
1706018400
1706022000
1706025600
1706029200
1706032800
1706036400
1706040000
1706043600
1706047200
1706050800
1706054400
1706058000
1706061600
1706065200
1706068800
1706072400
1706076000
1706079600
1706083200
1706086800
1706090400
1706094000
1706097600
1706101200
1706104800
1706108400
1706112000
1706115600
1706119200
1706122800
1706126400
1706130000
1706133600
1706137200
1706140800
1706144400
1706148000
1706151600
1706155200
1706158800
1706162400
1706166000
1706169600
1706173200
1706176800
1706180400
1706184000
1706187600
1706191200
1706194800
1706198400
1706202000
1706205600
1706209200
1706212800
1706216400
1706220000
1706223600
1706227200
1706230800
1706234400
1706238000
1706241600
1706245200
1706248800
1706252400
1706256000
1706259600
1706263200
1706266800
1706270400
1706274000
1706277600
1706281200
1706284800
1706288400
1706292000
1706295600
1706299200
1706302800
1706306400
1706310000
1706313600
1706317200
1706320800

In [117]:
import json
conn = sqlite3.connect('weather.db')
cursor = conn.cursor()

def extract_load(links):

    for link in links:

        response = requests.get(link)
        data = response.json()

        d1 = {'lat': data['lat'],
              'lon': data['lon'],
              'timezone': data['timezone'],
              'timezone_offset': data['timezone_offset']
              }

        for i in data['data']:
            d1.update(i)

        for i in d1['weather']:
            d1.update(i)

        d1.pop('weather')

        data = d1
        d2 = [data]
        cursor.executescript('''

                    CREATE TABLE IF NOT EXISTS weather (
                            
                            data json    
                )
            ''')

        cursor.execute('INSERT INTO weather VALUES (?)', d2)
        conn.commit()

In [114]:
for i in data.items():
    print(i)

('lat', 51.5072)
('lon', 0.1276)
('timezone', 'Europe/London')
('timezone_offset', 0)
('dt', 1708590535)
('sunrise', 1708585263)
('sunset', 1708622726)
('temp', 10.33)
('feels_like', 9.77)
('pressure', 986)
('humidity', 90)
('dew_point', 8.76)
('uvi', 0.04)
('clouds', 75)
('visibility', 10000)
('wind_speed', 5.66)
('wind_deg', 170)
('id', 500)
('main', 'Rain')
('description', 'light rain')
('icon', '10d')


In [118]:
extract_load(get_links())

ProgrammingError: Error binding parameter 1: type 'dict' is not supported

In [91]:
import pandas as pd

df = pd.read_sql_query('''SELECT * FROM weather''', conn)
df

,data
